In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [4]:
df_train = df_train.fillna("na")
df_test = df_test.fillna("na")

In [8]:
def text_preprocessing(text):
    from nltk.corpus import stopwords
    from nltk.tokenize import RegexpTokenizer
    from nltk.stem import WordNetLemmatizer
    
    tokenizer = RegexpTokenizer(r'\w+')
    stop = set(stopwords.words('english')) 
    lemmatizer = WordNetLemmatizer()
     
    processed = []   
    print("Start preprocessing: ")
    for i in range(len(text)):
        if i == 10:
            print("yehaa we making progress")
        if i%10000 == 0:
            print("{} of {}".format(i, len(text)))
        temp = tokenizer.tokenize(text.iloc[i])
        temp = [lemmatizer.lemmatize(i) for i in temp if i not in stop]
        temp = (" ").join(temp)
        processed.append(temp)
        
    return processed

In [12]:
df_train["processed"] = text_preprocessing(df_train["comment_text"])
df_test["processed"] = text_preprocessing(df_test["comment_text"])

Start preprocessing: 
0 of 95851
yehaa we making progress
10000 of 95851
20000 of 95851
30000 of 95851
40000 of 95851
50000 of 95851
60000 of 95851
70000 of 95851
80000 of 95851
90000 of 95851
Start preprocessing: 
0 of 226998
yehaa we making progress
10000 of 226998
20000 of 226998
30000 of 226998
40000 of 226998
50000 of 226998
60000 of 226998
70000 of 226998
80000 of 226998
90000 of 226998
100000 of 226998
110000 of 226998
120000 of 226998
130000 of 226998
140000 of 226998
150000 of 226998
160000 of 226998
170000 of 226998
180000 of 226998
190000 of 226998
200000 of 226998
210000 of 226998
220000 of 226998


In [14]:
df_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,processed
0,22256635,"Nonsense? kiss off, geek. what I said is true...",1,0,0,0,0,0,Nonsense kiss geek I said true I account termi...
1,27450690,"""\n\n Please do not vandalize pages, as you di...",0,0,0,0,0,0,Please vandalize page edit W S Merwin If conti...
2,54037174,"""\n\n """"Points of interest"""" \n\nI removed the...",0,0,0,0,0,0,Points interest I removed point interest secti...
3,77493077,Asking some his nationality is a Racial offenc...,0,0,0,0,0,0,Asking nationality Racial offence Wow aware Bl...
4,79357270,The reader here is not going by my say so for ...,0,0,0,0,0,0,The reader going say ethereal vocal style dark...


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import time

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', SGDClassifier(loss='log')),
])

parameters = {
    'tfidf__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000, 50000),
    'tfidf__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    #'tfidf__use_idf': (True, False),
    #'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
    #'clf__n_iter': (10, 50, 80),
}

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=10)

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
print(parameters)
t0 = time.time()
grid_search.fit(df_train["processed"], df_train["obscene"])
print("done in %0.3fs" % (time.time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
pipeline: ['tfidf', 'clf']
parameters:
{'tfidf__max_df': (0.5, 0.75, 1.0), 'tfidf__ngram_range': ((1, 1), (1, 2)), 'clf__alpha': (1e-05, 1e-06), 'clf__penalty': ('l2', 'elasticnet')}
Fitting 3 folds for each of 24 candidates, totalling 72 fits

[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done  20 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done  49 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done  60 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done  69 out of  72 | elapsed:  7.4min remaining:   19.3s
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:  7.8min finished



done in 515.503s

Best score: 0.979
Best parameters set:
	clf__alpha: 1e-05
	clf__penalty: 'elasticnet'
	tfidf__max_df: 0.75
	tfidf__ngram_range: (1, 2)


In [33]:
grid_search.best_params_

{'clf__alpha': 1e-05,
 'clf__penalty': 'elasticnet',
 'tfidf__max_df': 0.75,
 'tfidf__ngram_range': (1, 2)}

In [65]:
vec = TfidfVectorizer(#max_df = 0.75, 
                     ngram_range=(1,2))

clf = SGDClassifier(loss='log',
         alpha=1e-05,
         penalty= 'elasticnet')

tfidf_train = vec.fit_transform(df_train["processed"])
tfidf_test = vec.transform(df_test["processed"])
clf.fit(tfidf_train, df_train["obscene"])
clf.predict_proba(tfidf_test)

array([[ 0.97988535,  0.02011465],
       [ 0.97847339,  0.02152661],
       [ 0.99500372,  0.00499628],
       ..., 
       [ 0.98895814,  0.01104186],
       [ 0.96402706,  0.03597294],
       [ 0.99717545,  0.00282455]])

In [54]:
tfidf_test = vec.fit_transform(df_test["processed"])
clf.predict(tfidf_test)

ValueError: X has 3794912 features per sample; expecting 1943700

In [59]:
print(tfidf_train.shape)
print(tfidf_test.shape)
print(len(df_train))

(95851, 1943700)
(226998, 3794912)
95851


In [63]:
clf.predict_proba(tfidf_test)

ValueError: X has 3794912 features per sample; expecting 1943700